In [ ]:
import numpy
import sys
import pymzml
import scipy
import os
import math
import tensorly
import peakutils
import matplotlib
from scipy import signal
from matplotlib import pyplot
import glob
array3D = []
raw_array = []
raw_time = []
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
labels = []
for filename in sorted(glob.glob('D:/asp/extracts 2/test/*mzML'), key=numericalSort):    
    labels.append(filename.split('-')[1][:-5])
    run = pymzml.run.Reader(filename)
    array = []
    mass = []
    time = []
    for spectrum in run:
        time.append(round(spectrum.scan_time_in_minutes(),2))
        mass.append(spectrum.mz)
        peaks = spectrum.peaks("raw")[:,1]
        array.append(peaks)  
             
    #reduce dimensionality by mass, 
    #intensities for signals with mass defects in the range from -0,35 to +0,65 
    #were summarized and assigned to a cell with an integer m/z value 
    aligned_mass = numpy.zeros(((len(time)),1300))
    for i in range (len(time)):   
        for j in range (len(mass[i])):
            int_mass = int(math.modf(mass[i][j])[1]) 
            if mass[i][j] < (int_mass+0.65):
                aligned_mass[i][int_mass] = aligned_mass[i][int_mass] + array[i][j]      
            else:
                if int_mass < 1299:
                    aligned_mass[i][int_mass+1] = aligned_mass[i][int_mass+1] + array[i][j]
                else:
                    aligned_mass[i][int_mass] = aligned_mass[i][int_mass] + array[i][j]
    raw_array.append(aligned_mass)
    raw_time.append(time)
    
    
    #denoise each mass-chromatogram using Savitzki-Golay filter 
    corr_noise = numpy.zeros((1300,len(time)))
    raw_sim = numpy.zeros(len(time))
    for j in range (100,1300):
        corr_noise[j] = scipy.signal.savgol_filter(aligned_mass[:,j],window_length=13,polyorder=1)              
    al_time = numpy.arange(3.02,22.0,0.05)
    finaldata = numpy.zeros((380,1200))
    ar = []
    ar.append([])
    n = 0
    
    
    #time axis was rescaled with the step size 0.05 min in order to achieve unified range. 
    #Final time scale was cut and had a range from 2 to 22 minutes
        for j in range (len(time)):       
            if time[j] > 2.98 and time[j]<21.97:
                if time[j] <= al_time[n]:
                    ar[n].append(corr_noise[i][j])
                else:
                    ar.append([])
                    ar[n+1].append(corr_noise[i][j])
                    n = n+1
        for k in range (380):
            finaldata[k][i - 100]=max(ar[k])
        ar = []
        ar.append([])
        n = 0
    
    array3D.append(finaldata)
    
from tensorly.decomposition import parafac
final_tensor = numpy.array(array3D)